In [ ]:
import asf_search as asf
import os
import json
from shapely.geometry import shape

def download_sentinel1_simple(aoi_geojson_path, output_directory, start_date, end_date,
                              orbit_direction="ASC", max_results=5):
    """
    Simple Sentinel-1 downloader with custom AOI, output directory, dates, and orbit direction.
    orbit_direction: "ASC" for ascending, "DESC" for descending
    """
    
    # 1. Load GeoJSON AOI and convert to WKT
    with open(aoi_geojson_path, 'r') as f:
        geojson_data = json.load(f)
    
    if geojson_data['type'] == 'FeatureCollection':
        geometry_dict = geojson_data['features'][0]['geometry']
    else:
        geometry_dict = geojson_data['geometry']
    
    shapely_geom = shape(geometry_dict)
    geometry_wkt = shapely_geom.wkt
    
    print(f"✓ Loaded AOI from: {aoi_geojson_path}")
    
    # 2. Authenticate (Use environment variables or manually enter)
    username = "YOUR_USERNAME"
    password = "YOUR_PASSWORD"
    
    session = asf.ASFSession().auth_with_creds(username, password)
    print("✓ Authentication successful")
    
    # 3. Set orbit direction
    if orbit_direction.upper() == "ASC":
        flight_dir = asf.FLIGHT_DIRECTION.ASCENDING
    else:
        flight_dir = asf.FLIGHT_DIRECTION.DESCENDING
    
    # 4. ASF Search
    results = asf.search(
        platform=asf.PLATFORM.SENTINEL1,
        processingLevel=asf.PRODUCT_TYPE.GRD_HD,
        start=start_date,
        end=end_date,
        intersectsWith=geometry_wkt,
        beamMode=asf.BEAMMODE.IW,
        flightDirection=flight_dir,
        maxResults=max_results
    )
    
    print(f"✓ Found {len(results)} {orbit_direction} products")
    
    # 5. Download
    os.makedirs(output_directory, exist_ok=True)
    
    for product in results:
        try:
            product.download(path=output_directory, session=session)
            print(f"✓ Downloaded: {product.properties['fileName']}")
        except Exception as e:
            print(f"✗ Failed: {product.properties['fileName']} - {e}")
    
    print(f"Download complete! Files saved to: {output_directory}")


# ------------------------
# USER INPUT PLACEHOLDERS
# ------------------------

AOI_PATH = "PATH_TO_YOUR_AOI.geojson"
OUTPUT_DIR = "OUTPUT_DIRECTORY_FOR_DOWNLOADS"
START_DATE = "YYYY-MM-DD"
END_DATE = "YYYY-MM-DD"

# Example call:
# download_sentinel1_simple(AOI_PATH, OUTPUT_DIR, START_DATE, END_DATE, orbit_direction="ASC")